In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import osmnx as ox
from shapely.geometry import Point
from pyproj import Transformer
import osmnx.folium as ox_folium

sns.set_context(context='paper')

%matplotlib inline

https://wiki.openstreetmap.org/wiki/Map_features#Roads

In [3]:
lat, lon = 33.758351795782595, -84.38221086204965 #160-148 Ellis St NE, Atlanta, GA 30303
point = Point(lon, lat)

In [4]:
gdf_point = gpd.GeoDataFrame([{'geometry': point}], crs="EPSG:4326")

In [5]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857")
x, y = transformer.transform(lat, lon)

In [10]:
gdf_point_proj = gdf_point.to_crs(epsg=3857)
gdf_buffer = gdf_point_proj.buffer(500) #distance

In [11]:
gdf_buffer = gdf_buffer.to_crs(epsg=4326)
bbox = gdf_buffer.total_bounds

In [12]:
G = ox.graph_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], network_type='all')

desired_highway_types = ["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "residential", "living_street", "pedestrian"]

edges_to_remove = [
    (u, v, k) for u, v, k, data in G.edges(keys=True, data=True)
    if "highway" not in data or not any(hwy_type in data["highway"] for hwy_type in desired_highway_types)
]
G.remove_edges_from(edges_to_remove)

ctx = ox_folium.plot_graph_folium(G, popup_attribute='highway', tiles='openstreetmap')

ctx

C:\Users\synnp\AppData\Local\Temp\ipykernel_17364\1500332001.py:1: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.graph_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], network_type='all')
C:\Users\synnp\AppData\Local\Temp\ipykernel_17364\1500332001.py:1: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = ox.graph_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], network_type='all')
C:\Users\synnp\AppData\Local\Temp\ipykernel_17364\1500332001.py:11: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.

In [13]:
gdf_roads = ox.utils_graph.graph_to_gdfs(G, nodes=False)
gdf_roads

C:\Users\synnp\AppData\Local\Temp\ipykernel_17364\656890413.py:1: FutureWarning: The `graph_to_gdfs` function has moved to the `convert` module. Calling `utils_graph.graph_to_gdfs` is deprecated and will be removed in the v2.0.0 release. Call it via `convert.graph_to_gdfs` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf_roads = ox.utils_graph.graph_to_gdfs(G, nodes=False)


osmid  \
u           v           key                                              
69100812    339264283   0            [507036882, 953034628, 953034629]   
69108378    463530185   0                          [38970425, 9235148]   
            69433134    0    [38945232, 39169057, 39169032, 337901420]   
69112401    69112713    0                        [38974050, 175629535]   
            69121345    0                                   1111331909   
...                                                                ...   
11883147550 11883147553 0                                   1280112739   
11883147553 1811042767  0                                   1280112739   
11955679698 5415792784  0                                    989914212   
12058417552 1811042767  0                                      9241753   
            69179054    0                                      9241753   

                             oneway   lanes                           name  \
u           v           key                                                  
69100812    339264283   0      True  [5, 4]      Piedmont Avenue Northeast   
69108378    463530185   0      True       2                            NaN   
            69433134    0      True  [5, 6]             Downtown Connector   
69112401    69112713    0      True       2                            NaN   
            69121345    0      True       2                            NaN   
...                             ...     ...                            ...   
11883147550 11883147553 0      True     NaN                            NaN   
11883147553 1811042767  0      True     NaN                            NaN   
11955679698 5415792784  0     False     NaN        Jesse Hill Junior Drive   
12058417552 1811042767  0     False     NaN  Baker Highland Conn Northeast   
            69179054    0     False     NaN  Baker Highland Conn Northeast   

                                    highway  reversed   length  \
u           v           key                                      
69100812    339264283   0         secondary     False   89.611   
69108378    463530185   0     motorway_link     False  298.397   
            69433134    0          motorway     False  450.015   
69112401    69112713    0        trunk_link     False   69.473   
            69121345    0        trunk_link     False   54.592   
...                                     ...       ...      ...   
11883147550 11883147553 0    secondary_link     False   35.079   
11883147553 1811042767  0    secondary_link     False   28.420   
11955679698 5415792784  0       residential     False   54.095   
12058417552 1811042767  0         secondary     False    8.246   
            69179054    0         secondary      True   93.130   

                                                                      geometry  \
u           v           key                                                      
69100812    339264283   0    LINESTRING (-84.38216 33.76102, -84.38215 33.7...   
69108378    463530185   0    LINESTRING (-84.37824 33.75565, -84.37816 33.7...   
            69433134    0    LINESTRING (-84.37824 33.75565, -84.37833 33.7...   
69112401    69112713    0    LINESTRING (-84.37863 33.75990, -84.37892 33.7...   
            69121345    0    LINESTRING (-84.37863 33.75990, -84.37874 33.7...   
...                                                                        ...   
11883147550 11883147553 0    LINESTRING (-84.37798 33.76180, -84.37802 33.7...   
11883147553 1811042767  0    LINESTRING (-84.37816 33.76153, -84.37825 33.7...   
11955679698 5415792784  0    LINESTRING (-84.38028 33.75464, -84.38026 33.7...   
12058417552 1811042767  0    LINESTRING (-84.37849 33.76138, -84.37840 33.7...   
            69179054    0    LINESTRING (-84.37849 33.76138, -84.37866 33.7...   

                            bridge        ref maxspeed access  
u           v           key                                    
69100812    339264283   0      Na